# Final Capstone

In this notebook we load ~10MM rows of data from the Kaggle Avazu click prediction competition.  We reduce this to 2MM rows for speed. We then create train/test splits, perform feature engineering, and then make predictions which we validate with our test split of data. Similar to the Kaggle competition we use Log Loss as our metric for evaluation of the model's performance.

With the exception of a date column ('hour'), all of the columns in this data set are categorical, even though some are encoded as integers (source: Kaggle docs and boards).  After 'id' and 'hour' are dropped, therefore, all values are converted to strings, so that every column can be treated with a separate CountVectorizer which is fit on training data and which performs transformations on both training and test data.  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import resample
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from scipy import sparse
from sklearn.feature_extraction import FeatureHasher 
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost
from scipy.sparse import hstack
from sklearn.metrics import classification_report, log_loss, roc_auc_score, roc_curve, confusion_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier

/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Loading 10MM rows
%time train = pd.read_csv("../assets/trainaa")

CPU times: user 40.9 s, sys: 6.06 s, total: 47 s
Wall time: 48.3 s


In [3]:
print(f"Shape of train before sampling: {train.shape}")

Shape of train before sampling: (9999999, 24)


In [4]:
# Taking 2MM random rows
train = train.sample(frac=0.2, replace=False)

In [5]:
print(f"Shape of train after sampling: {train.shape}")

Shape of train after sampling: (2000000, 24)


In [6]:
# Dropping un-needed columns
train = train.drop(columns=['hour', 'id'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=['click']), train['click'], random_state=99, stratify=train['click'])

In [8]:
X_train.columns

Index(['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id',
       'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model',
       'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18',
       'C19', 'C20', 'C21'],
      dtype='object')

In [9]:
# From EDA we know there's a negative value here 
X_train.C20.replace(to_replace=-1, value=1, inplace=True)
X_test.C20.replace(to_replace=-1, value=1, inplace=True)

In [10]:
# two banner dimensions combined into one feature
X_train['banner'] = X_train['C15'].map(str) + X_train['C16'].map(str)
X_test['banner'] = X_test['C15'].map(str) + X_test['C16'].map(str)

In [11]:
X_train = X_train.drop(columns=['C15', 'C16'])
X_test = X_test.drop(columns=['C15', 'C16'])

In [ ]:
# Convert all values to strings since these are all categorical columns
X_train = X_train.applymap(str)
X_test = X_test.applymap(str)

In [ ]:
columns = X_train.columns.values.tolist()

In [ ]:
train_vec = []
test_vec = []

In [ ]:
# create a vectorized representation of all columns, while ensuring congruent shapes between train and test

for col in columns:
    cvec = CountVectorizer(stop_words=None, min_df=1, token_pattern=r"(?u)\b\w+\b")
    train_transform = cvec.fit_transform(X_train[col])
    test_transform = cvec.transform(X_test[col])
    train_vec.append(train_transform)
    test_vec.append(test_transform)
    print(f"completed {col}. train shape {train_transform.shape} - test shape {test_transform.shape}")


In [ ]:
train_matrix = sparse.hstack(train_vec)

In [ ]:
test_matrix = sparse.hstack(test_vec)

In [ ]:
print(f"Train shape: {train_matrix.shape} Test shape: {test_matrix.shape}")

In [ ]:
results = pd.DataFrame(y_test)

In [ ]:
nb = MultinomialNB()
nb.fit(train_matrix, y_train)
y_pred_nb = nb.predict(test_matrix)
results['NB'] = y_pred_nb

In [ ]:
log_loss(y_test, y_pred_nb)

In [ ]:
print(classification_report(y_test, y_pred_nb))

In [ ]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(train_matrix, y_train)
y_pred_lr = lr.predict(test_matrix)
results['LR'] = y_pred_lr
log_loss(y_test, y_pred_lr)

In [ ]:
print(classification_report(y_test, y_pred_lr))

In [ ]:
mlp = MLPClassifier()
mlp.fit(train_matrix, y_train)
y_pred_mlp = mlp.predict(test_matrix)
results['MLP'] = y_pred_mlp
log_loss(y_test, y_pred_mlp)

In [ ]:
print(classification_report(y_test, y_pred_mlp))

In [ ]:
results['AVG'] = (results['NB'] + results['MLP'] + results['LR'])/3

In [ ]:
log_loss(y_test, results['AVG'])

In [ ]:
print(classification_report(y_test, results['AVG']))

In [ ]:
rf = RandomForestClassifier(class_weight='balanced')
rf.fit(train_matrix, y_train)
y_pred_rf = rf.predict(test_matrix)
results['RF'] = y_pred_rf
log_loss(y_test, y_pred_rf)

In [ ]:
print(classification_report(y_test, y_pred_rf))

In [ ]:
results['AVG'] = (results['NB'] + results['MLP'] + results['LR'] + results['RF'])/4
log_loss(y_test, results['AVG'])

In [ ]:
print(classification_report(y_test, results['AVG']))